In [29]:
# ------------------ 1. ⚙️ Install Dependencies ------------------
%pip install --quiet --upgrade langchain langchain-community langchain-openai langchain-experimental neo4j pandas requests


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# ------------------ 2. 🔐 Environment Setup ------------------
import os
from dotenv import load_dotenv


# 🧹 Clear existing environment variables
os.environ.pop("NEO4J_URI", None)
os.environ.pop("NEO4J_USERNAME", None)
os.environ.pop("NEO4J_PASSWORD", None)

load_dotenv()

# 🗝️ Set environment variables
openai_api_key = os.environ["OPENAI_API_KEY"]
NEO4J_URI = os.environ["NEO4J_URI"]
NEO4J_USERNAME= os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD= os.environ["NEO4J_PASSWORD"]



print("Environment variables loaded successfully.")

# ------------------ 3. 📂 File Paths ------------------

csv_path = os.path.abspath(os.path.join(os.getcwd(), "..", "Data", "movie_data.csv"))

Environment variables loaded successfully.


In [3]:
# ------------------ 4. 🚀 Ingest Data into Neo4j ------------------
from neo4j import GraphDatabase
import pandas as pd
from tqdm import tqdm

class NetflixGraphDB:
    def __init__(self, NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD):
        self.driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        print("✅ Connected to Neo4j")

    def close(self):
        self.driver.close()
        print("🔌 Connection closed")

    def clear_database(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
            print("🧹 Cleared entire graph")

    def load_movies_from_csv(self, csv_path):
        df = pd.read_csv(csv_path)
        df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    
        print(f"📄 Loading {len(df)} movie rows from: {csv_path}")

        with self.driver.session() as session:
            for _, row in tqdm(df.iterrows(), total=len(df), desc="🚀 Ingesting movies"):
                row = row.where(pd.notnull(row), None)

                title = row.get("title")
                release_year = row.get("release_year")
                duration = row.get("duration")
                rating = row.get("rating")
                description = row.get("description")
                show_type = row.get("type")
                country = row.get("country")
                genres = row.get("listed_in")
                date_added = row.get("date_added")

                if not title:
                    print("⚠️ Skipping movie with missing title.")
                    continue

            # Movie node
                session.run(
                """
                MERGE (m:Show {title: $title, release_year: $release_year})
                SET m.duration = $duration,
                    m.rating = $rating,
                    m.description = $description,
                    m.date_added = $date_added,
                    m.type = $type
                """,
                {
                    "title": title,
                    "release_year": release_year,
                    "duration": duration,
                    "rating": rating,
                    "description": description,
                    "date_added": date_added,
                    "type": show_type
                }
            )

            # Country relationship
                if country:
                    session.run(
                    """
                    MERGE (c:Country {name: $country})
                    WITH c
                    MATCH (m:Show {title: $title, release_year: $release_year})
                    MERGE (m)-[:PRODUCED_IN]->(c)
                    """,
                    {"country": country, "title": title, "release_year": release_year}
                )

            # Duration as a separate node (optional)
                if duration:
                    session.run(
                    """
                    MERGE (d:Duration {label: $duration})
                    WITH d
                    MATCH (m:Show {title: $title, release_year: $release_year})
                    MERGE (m)-[:HAS_DURATION]->(d)
                    """,
                    {"duration": duration, "title": title, "release_year": release_year}
                )

            # Genres
                if genres:
                    for genre in str(row.get("listed_in", "")).split(","):
                        genre = genre.strip().title()
                        if genre:
                         session.run(
                            """
                            MERGE (g:Genre {name: $genre})
                            WITH g
                            MATCH (m:Show {title: $title, release_year: $release_year})
                            MERGE (m)-[:IN_GENRE]->(g)
                            """,
                            {"genre": genre, "title": title, "release_year": release_year}
                        )

            # Directors
                for director in str(row.get("director", "")).split(","):
                    director = director.strip()
                    if director:
                        session.run(
                        """
                        MERGE (p:Person {name: $name})
                        WITH p
                        MATCH (m:Show {title: $title, release_year: $release_year})
                        MERGE (p)-[:DIRECTED]->(m)
                        """,
                        {"name": director, "title": title, "release_year": release_year}
                    )

            # Cast
                for actor in str(row.get("cast", "")).split(","):
                    actor = actor.strip()
                    if actor:
                        session.run(
                        """
                        MERGE (p:Person {name: $name})
                        WITH p
                        MATCH (m:Show {title: $title, release_year: $release_year})
                        MERGE (p)-[:ACTED_IN]->(m)
                        """,
                        {"name": actor, "title": title, "release_year": release_year}
                    )

    print("✅ All data ingested into Neo4j.")



✅ All data ingested into Neo4j.


In [25]:
# ------------------ 4. 🔗 Connect to Neo4j ------------------
# Create an instance of NetflixGraphDB
db = NetflixGraphDB(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

db.clear_database()  # Optional: wipe old data
db.load_movies_from_csv(csv_path)

# Always close when done
db.close()

✅ Connected to Neo4j
🧹 Cleared entire graph
📄 Loading 390 movie rows from: /home/prashant-agrawal/Netflix_Project/Data/movie_data.csv


🚀 Ingesting movies: 100%|██████████| 390/390 [05:16<00:00,  1.23it/s]

🔌 Connection closed


In [4]:
# ------------------ 3. 🧠 Load LLM (OpenAI GPT-4o) ------------------
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [ ]:
# ------------------ 4. 🔗 Connect to Neo4j ------------------
from langchain_community.graphs import Neo4jGraph  # 👈 compatible with GraphCypherQAChain
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
from langchain.prompts import ChatPromptTemplate

cypher_prompt = ChatPromptTemplate.from_template("""
You are a Cypher query generation assistant.

Your ONLY job is to return a Cypher query. Do not include explanations, markdown, comments, or natural language.

---

Schema:
{schema}

---

📌 General Rules:
- Use node label `Show` for all movies. Always include `s.type = "Movie"` in the WHERE clause.
- Never use a non-existent label like `Movie`.
- Do **NOT** hallucinate future years (no 2023 or beyond).
- Use `toLower(g.name) CONTAINS '<genre>'` to filter genres.
- For recency, sort by: `ORDER BY s.release_year DESC`.
- If the user gives no count, default to `LIMIT 15`.
- If the user gives a number (e.g. "top 50", "100 movies"), ALWAYS use that number in the `LIMIT`.
- Never reduce the requested limit. If the user says "100 movies", return a Cypher with `LIMIT 100` — not less.
- Be strict: only generate one Cypher query.

---

📌 Few-shot Examples:

User Question: "Show me the top 100 romantic movies"
Cypher:
MATCH (s:Show)-[:IN_GENRE]->(g:Genre)
WHERE s.type = "Movie" AND toLower(g.name) CONTAINS "romantic"
RETURN s.title, s.release_year
ORDER BY s.release_year DESC
LIMIT 100

---

User Question: "List the latest horror movie from India"
Cypher:
MATCH (s:Show)-[:IN_GENRE]->(g:Genre), (s)-[:PRODUCED_IN]->(c:Country)
WHERE s.type = "Movie" AND c.name = "India" AND toLower(g.name) CONTAINS "horror"
RETURN s.title, s.release_year
ORDER BY s.release_year DESC
LIMIT 1

---

User Question: "Give me 25 Indian comedy films released before 2015"
Cypher:
MATCH (s:Show)-[:IN_GENRE]->(g:Genre), (s)-[:PRODUCED_IN]->(c:Country)
WHERE s.type = "Movie" AND toLower(g.name) CONTAINS "comedy" AND c.name = "India" AND s.release_year < 2015
RETURN s.title, s.release_year
ORDER BY s.release_year DESC
LIMIT 25

---

User Question:
{question}

Cypher:
""")
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    timeout=60
)

# ------------------ 5. 🔗 Create GraphCypherQAChain ------------------
chain = GraphCypherQAChain.from_llm(
   llm=llm,
   top_k=100,
   graph=graph,
   verbose=True,
   enhanced_schema=True,
   allow_dangerous_requests=True,
   return_intermediate_steps=True,
   cypher_prompt=cypher_prompt,
)



In [32]:
import pandas as pd
import os

def save_neo4j_results_to_csv(results: list[dict], output_path: str = "neo4j_results.csv") -> str:
    """
    Extracts movie data from Neo4j GraphCypherQAChain results and saves it to a CSV file.
    
    Args:
        results (list): A list of dicts returned from a Cypher query like MATCH... RETURN s.
        output_path (str): File path to save the CSV. Default is "neo4j_results.csv".

    Returns:
        str: Path to the saved CSV file.
    """

    if not results:
        raise ValueError("Empty result set. No data to save.")

    # Extract 's' node properties if wrapped like {'s': {title: ..., release_year: ..., ...}}
    try:
        flattened = [entry['s'] if 's' in entry else entry for entry in results]
    except Exception as e:
        raise ValueError(f"Malformed data: {e}")

    # Convert to DataFrame
    df = pd.DataFrame(flattened)

    # Optional: sort columns if needed
    preferred_order = ['title', 'release_year', 'rating', 'duration', 'description', 'type', 'date_added']
    df = df[[col for col in preferred_order if col in df.columns] + 
            [col for col in df.columns if col not in preferred_order]]

    # Clean file path
    output_path = os.path.abspath(output_path)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save to CSV
    df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"✅ Saved {len(df)} entries to: {output_path}")
    return output_path


In [44]:

# Define the user question
import re


user_question = "List 200 movies"

# Extract top_k dynamically (you said no helper, so do inline)
match = re.search(r"\b(\d{2,4})\b", user_question)
top_k = int(match.group(1)) if match else 15

# Override top_k just-in-time
chain.top_k = top_k
print(f"🔍 Searching for top {chain.top_k} movies...")

🔍 Searching for top 200 movies...


In [45]:

response = chain.invoke({"query": user_question})

# Extract query and context
cypher_query = response["intermediate_steps"][0].get("query", "N/A")
context_data = response["intermediate_steps"][1].get("context", [])

print("Cypher query:", cypher_query)
print("Returned rows:", len(context_data))

# Optional: Convert to DataFrame and preview
import pandas as pd
df = pd.DataFrame(context_data)
print(df.head())
df.to_csv("exports/indian_horror_movies.csv", index=False)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Show)
WHERE s.type = "Movie"
RETURN s.title, s.release_year
ORDER BY s.release_year DESC
LIMIT 200
Full Context:
[{'s.title': '18 Presents', 's.release_year': 2020}, {'s.title': 'All The Bright Places', 's.release_year': 2020}, {'s.title': 'All Together Now', 's.release_year': 2020}, {'s.title': 'A Trash Truck Christmas', 's.release_year': 2020}, {'s.title': 'An Unremarkable Christmas', 's.release_year': 2020}, {'s.title': '365 Days', 's.release_year': 2020}, {'s.title': 'A Whisker Away', 's.release_year': 2020}, {'s.title': 'Òlòtūré', 's.release_year': 2020}, {'s.title': 'A Go! Go! Cory Carson Christmas', 's.release_year': 2020}, {'s.title': 'Ala Vaikunthapurramuloo', 's.release_year': 2020}, {'s.title': 'A Secret Love', 's.release_year': 2020}, {'s.title': 'Altered Carbon: Resleeved', 's.release_year': 2020}, {'s.title': 'A Life of Speed: The Juan Manuel Fangio Story', 's.release_year': 2020}, {'s.title': 'All Be